# Automatic question categorization (modelisation)
Pierre-Yves BOISBUNON - February 2018

----------

In this notebook, we will continue the categorization job to establish classification models.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from __future__ import division
import utils

# 1 Import clean dataset

First open previously stored cleaned dataset:

In [2]:
df = pd.read_csv('dataset_clean.csv', sep='\t')
df.head()

,index,Body,Title,Tags,y,raw,raw_letter
0,0,<p>I have two dataframes that I'm trying to co...,How to combine two differently multi-indexed p...,python|pandas,python pandas,I have two dataframes that I'm trying to combi...,have two dataframes that try to combine they e...
1,2,"<p>I have Eclipse CDT Oxygen on Ubuntu 16.04, ...",Eclipse CDT Oxygen with LLVM support forces st...,c++|llvm|ubuntu-16.04|eclipse-cdt|clang++,c++,"I have Eclipse CDT Oxygen on Ubuntu 16.04, wit...",have eclipse cdt oxygen on ubuntu with the llv...
2,3,<p>I use ' SelectionMode : Row ' now</p>\n\n<p...,How to get selected row in C1flexgrid?,c#|c1flexgrid,c#,I use ' SelectionMode : Row ' now\n\nI already...,use selectionmode row now already use mousecli...
3,5,"<p>So I have 24 ""person"" objects, which I crea...","Javascript, map returns undefined",javascript|arrays|object|dictionary,javascript arrays,"So I have 24 ""person"" objects, which I created...",so have person object which create use name ar...
4,6,<p>I want to extract only jms message text wit...,Extract jms text content,java|jms|message-queue,java,I want to extract only jms message text withou...,want to extract only jms message text without ...


Let's import cleanly **y** column.

In [3]:
df['y'] = df['y'].apply(lambda x: x.split())
df['y'][:5]

0        [python, pandas]
1                   [c++]
2                    [c#]
3    [javascript, arrays]
4                  [java]
Name: y, dtype: object

Let's define a **random state** in order models are computed with the same values.

In [4]:
# Random state in order to force computation with the same results
random_state = 100

## 1.1 Utils functions

Let's define the  **plot_learning_curve** function used later in the notebook:

In [5]:
from sklearn.model_selection import learning_curve
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5),
                       scoring=None):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes,
        scoring=scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

Let's define the **EstimatorSelectionHelper** class used for cross validation:

In [6]:
from sklearn.model_selection import GridSearchCV

class EstimatorSelectionHelper:
    """
    Custom class for cross validation computation.
    It takes as parameters:
    - models and their associated parameters
    """
    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}
    
    def fit(self, X, y, cv=5, n_jobs=1, verbose=1, scoring=None):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs, 
                              verbose=verbose, scoring=scoring)
            gs.fit(X,y)
            self.grid_searches[key] = gs   
                
    def score_summary(self, sort_by='mean_score'):
        def row(key, train_score, test_score, std_test_score, params):
            d = {
                 'estimator': key,
                 'train_score': train_score,
                 'test_score': test_score,
                 'std_test_score': std_test_score * 2
            }
            d_copy = d.copy()
            d_copy.update(params)
            return pd.Series(d_copy)
                      
        rows = [row(k, train_score, test_score, std_test_score, params) 
                    for k in helper.keys
                     for train_score, test_score, std_test_score, params 
                        in zip(self.grid_searches[k].cv_results_['mean_train_score'],
                               self.grid_searches[k].cv_results_['mean_test_score'], 
                               self.grid_searches[k].cv_results_['std_test_score'],
                               self.grid_searches[k].cv_results_['params'])]
        df = pd.concat(rows, axis=1).T#.sort([sort_by], ascending=False)
        
        columns = ['estimator', 'train_score', 'test_score', 'std_test_score']
        columns = columns + [c for c in df.columns if c not in columns]
        
        return df[columns]

Let's define our custom scorer based on **f1 macro** computation: metrics used for classification.

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_recall_fscore_support

def tag_scorer(y_test,y_predict):
    """
    Function defining custom scorer based on f1 macro metric
    """
    f1_score = precision_recall_fscore_support(y_test, y_predict, average = 'macro')[2]
    return f1_score

# 2 Supervised method for tags prediction

For the modelisation, we wil use three different classifiers:
- LinearSVC classifier
- Random Forest classifier
- XGBoost classifier


Let's first optimize optimize feature selection

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

## 2.1 Optimize feature selections

In [9]:
# Create X and y vectors
y = df['y'].tolist()
X = df["raw_letter"]

# Binarize y
print("Binarize y")
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_trans = mlb.fit_transform(y)

# Bag of word vectorizer
print("Bag of words vectorizer")
vect = CountVectorizer(stop_words='english', 
                       lowercase=True, 
                       tokenizer=lambda x: x.split(' '),
                       max_df=0.2, min_df=100)
tfidf = TfidfTransformer(use_idf=True, norm='l2')
pipeline_vect_tfidf = Pipeline([
    ('vect', vect),
    ('tfidf', tfidf)
])
X_tf = pipeline_vect_tfidf.fit_transform(X)

# Create train/test patterns
print("Create train/test patterns")
X_train, X_test, y_train, y_test = train_test_split(X_tf, y_trans,
                                                    random_state=random_state, 
                                                    test_size=0.2)


Binarize y
Bag of words vectorizer
Create train/test patterns


## 2.2 LinearSVC

Let's start first with **LinearSVC** model.

In [10]:
# Model classifier
clf = OneVsRestClassifier(estimator=LinearSVC(random_state=random_state, dual=False, penalty='l1'))

pipeline = Pipeline([
    ('clf', clf)
])

parameters = {
    'clf__estimator__penalty': ('l1', 'l2'),
    'clf__estimator__dual': [False]
}

models = { 
    'OVR_SVC': pipeline
}

params = { 
    'OVR_SVC': [parameters]
}

helper = EstimatorSelectionHelper(models, params)
helper.fit(X_train, y_train, n_jobs=-1, scoring='f1_macro')
model_result = helper.score_summary(sort_by='min_score')
model_result

Running GridSearchCV for OVR_SVC.
Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.6min finished


,estimator,train_score,test_score,std_test_score,clf__estimator__dual,clf__estimator__penalty
0,OVR_SVC,0.772902,0.69956,0.00836339,False,l1
1,OVR_SVC,0.823042,0.689399,0.00890963,False,l2


In [11]:
from sklearn.metrics import f1_score
y_predict = helper.grid_searches['OVR_SVC'].predict(X_test)
y_predict = np.array(y_predict)
f1_score(y_predict, y_test, average='macro')

0.7052799800691859

The best hyperparameter found is penality/dual equal to **l1/False** with a test score equal to **0.70**

## 2.3 RandomForest Classifier

In order to optimize Random Forest model, we will optimize the model in 3 steps:
- Step 1: Setup number of estimators.
- Step 2: Setup best split point.
- Step 3: Setup max_features

### 2.3.1 Tune n_estimators

Let's start tuning **n_estimators**

In [12]:
model_rfc = RandomForestClassifier(random_state=random_state)

clf = OneVsRestClassifier(estimator=model_rfc, n_jobs=-1)

pipeline = Pipeline([
    ('clf', clf)
])

parameters = {
    'clf__estimator__n_estimators': [50,100,200]
}

models = { 
    'OVR_RF': pipeline
}

params = { 
    'OVR_RF': [parameters]
}

helper = EstimatorSelectionHelper(models, params)
helper.fit(X_train, y_train, n_jobs=-1, scoring='f1_macro')
model_result = helper.score_summary(sort_by='min_score')
model_result

Running GridSearchCV for OVR_RF.
Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 109.9min finished


,estimator,train_score,test_score,std_test_score,clf__estimator__n_estimators
0,OVR_RF,0.998565,0.615273,0.0112692,50
1,OVR_RF,0.99988,0.622283,0.00899853,100
2,OVR_RF,0.999998,0.622077,0.0037976,200


Let's keep a number of estimators of **50** as it seems there is not a lot of difference in the test_score.
We can observe also a huge level of overfit.

### 2.3.2 Tune min_samples_split and min_samples_leaf

Let's try to optimize min_samples_split/min_samples_leaf to reduce the overfit.

In [13]:
model_rfc = RandomForestClassifier(random_state=random_state, n_estimators=50)

clf = OneVsRestClassifier(estimator=model_rfc, n_jobs=-1)

pipeline = Pipeline([
    ('clf', clf)
])

parameters = {
    'clf__estimator__min_samples_split': [2, 3, 4, 5, 6, 7],
    'clf__estimator__min_samples_leaf': [3, 4, 5]
}

models = { 
    'OVR_RF': pipeline
}

params = { 
    'OVR_RF': [parameters]
}

helper = EstimatorSelectionHelper(models, params)
helper.fit(X_train, y_train, n_jobs=-1, scoring='f1_macro')
model_result = helper.score_summary(sort_by='min_score')
model_result

Running GridSearchCV for OVR_RF.
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 108.1min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 224.7min finished


,estimator,train_score,test_score,std_test_score,clf__estimator__min_samples_leaf,clf__estimator__min_samples_split
0,OVR_RF,0.738917,0.560742,0.0110107,3,2
1,OVR_RF,0.738917,0.560742,0.0110107,3,3
2,OVR_RF,0.738917,0.560742,0.0110107,3,4
3,OVR_RF,0.738917,0.560742,0.0110107,3,5
4,OVR_RF,0.738917,0.560742,0.0110107,3,6
5,OVR_RF,0.735258,0.559425,0.00357937,3,7
6,OVR_RF,0.656381,0.530978,0.0115247,4,2
7,OVR_RF,0.656381,0.530978,0.0115247,4,3
8,OVR_RF,0.656381,0.530978,0.0115247,4,4
9,OVR_RF,0.656381,0.530978,0.0115247,4,5


Nice! We reduce the overfit. The best couple for min_samples_split/min_samples_leaf seems to be **3/6**.

### 2.3.3 Tune max_features

Let's tune now the max_features to optimize the test_score.

In [14]:
model_rfc = RandomForestClassifier(random_state=random_state, 
                                   n_estimators=50,
                                   min_samples_split=3,
                                   min_samples_leaf=6)

clf = OneVsRestClassifier(estimator=model_rfc, n_jobs=-1)

pipeline = Pipeline([
    ('clf', clf)
])

parameters = {
    'clf__estimator__max_features': ['auto', 'sqrt', 'log2', None]
}

models = { 
    'OVR_RF': pipeline
}

params = { 
    'OVR_RF': [parameters]
}

helper = EstimatorSelectionHelper(models, params)
helper.fit(X_train, y_train, n_jobs=-1, scoring='f1_macro')
model_result = helper.score_summary(sort_by='min_score')
model_result

Running GridSearchCV for OVR_RF.
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 430.9min finished


,estimator,train_score,test_score,std_test_score,clf__estimator__max_features
0,OVR_RF,0.55492,0.482247,0.0134742,auto
1,OVR_RF,0.55492,0.482247,0.0134742,sqrt
2,OVR_RF,0.151034,0.125634,0.00454774,log2
3,OVR_RF,0.816664,0.717229,0.00772181,None


In [15]:
from sklearn.metrics import f1_score
y_predict = helper.grid_searches['OVR_RF'].predict(X_test)
y_predict = np.array(y_predict)
f1_score(y_predict, y_test, average='macro')

0.7285733770054947

Finaly, if we observe the model is correctly optimized reducing at the best the overfit.
Let's try to optimize the test_score, specifiy a threshold of probability of the prediction of the RF model.

In [16]:
y_proba = helper.grid_searches['OVR_RF'].predict_proba(X_test)
y_proba = np.array(y_proba)
from sklearn.metrics import f1_score
threshold = [0.2,0.3,0.4,0.5]
score = []
for thresh in threshold:
    y_predict = np.zeros((y_proba.shape[0], y_proba.shape[1]))
    for i in range(0, y_proba.shape[0]):
        for j in range(0, len(y_proba[i,:])):
            #y_predict[i,j] = 0
            if (y_proba[i,j] > thresh):
                y_predict[i,j] = 1
    score.append(f1_score(y_predict, y_test, average='macro'))
score

[0.7235358018996912,
 0.7392861492575606,
 0.7394146609302844,
 0.7285733770054947]

A threshold equal to **0.4** gives the best validation score.

## 2.5 XGBoost

In order to optimize XGBoost model, we will optimize the model in 3 steps:
- Step 1: Setup number of estimators.
- Step 2: Setup max depth (node number) and min_child_weight.
- Step 3: Setup learning rate.

### 2.2.1 Tuning n_estimators

Let's tune first the number of estimators.

In [17]:
import xgboost as xgb

# See http://xgboost.readthedocs.io/en/latest/parameter.html
estimator = xgb.XGBClassifier(
              nthread=4,
              objective='multi:softprob',
              learning_rate=0.3,
              silent=1, 
              num_class=30,
              seed=random_state)

model_rfc = OneVsRestClassifier(estimator=estimator, n_jobs=-1)

pipeline = Pipeline([
    ('clf', model_rfc)
])

parameters = {
    'clf__estimator__n_estimators': [10, 50, 100]
}

models = { 
    'OVR_XGB': pipeline
}

params = { 
    'OVR_XGB': [parameters]
}

helper = EstimatorSelectionHelper(models, params)
helper.fit(X_train, y_train, n_jobs=-1, scoring='f1_macro')
model_result = helper.score_summary(sort_by='min_score')
model_result

Running GridSearchCV for OVR_XGB.
Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 411.5min finished


,estimator,train_score,test_score,std_test_score,clf__estimator__n_estimators
0,OVR_XGB,0.719879,0.683509,0.00749604,10
1,OVR_XGB,0.827344,0.722333,0.0081886,50
2,OVR_XGB,0.880932,0.727059,0.00947631,100


As there is not a lor of differences between 50 and 100 estimators, let's choose **50** estimators that should help in CPU calculation.

### 2.2.2 Tune max_depth and min_child_weight

Let's try to optimize validation score and reduce the overfit, optimizing max_depth and min_child_weight:

In [ ]:
import xgboost as xgb

# See http://xgboost.readthedocs.io/en/latest/parameter.html
estimator = xgb.XGBClassifier(
              nthread=4,
              objective='multi:softprob',
              learning_rate=0.3,
              silent=1, 
              num_class=30,
              n_estimators=50,
              seed=random_state)

model_xgb = OneVsRestClassifier(estimator=estimator, n_jobs=-1)

pipeline = Pipeline([
    ('clf', model_xgb)
])

parameters = {
    'clf__estimator__max_depth': [4,6,8],
    'clf__estimator__min_child_weight':[1,4]
}

models = { 
    'OVR_XGB': pipeline
}

params = { 
    'OVR_XGB': [parameters]
}

helper = EstimatorSelectionHelper(models, params)
helper.fit(X_train, y_train, n_jobs=-1, scoring='f1_macro')
model_result = helper.score_summary(sort_by='min_score')
model_result

Running GridSearchCV for OVR_XGB.
Fitting 5 folds for each of 6 candidates, totalling 30 fits


We can see that the best **max_depth** / **min_child_weight** couple is **4/4**, it gives the best cross validation score.

### 2.2.6 Tune learning_rate

Let's try to optimize also learning_rate:

In [ ]:
import xgboost as xgb

# See http://xgboost.readthedocs.io/en/latest/parameter.html
estimator = xgb.XGBClassifier(
              nthread=4,
              objective='multi:softprob',
              learning_rate=0.3,
              silent=1, 
              num_class=30,
              n_estimators=50,
              max_depth=4,
              min_child_weight=4,
              seed=random_state)

model_xgb = OneVsRestClassifier(estimator=estimator, n_jobs=-1)

pipeline = Pipeline([
    ('clf', model_xgb)
])

parameters = {
    'clf__estimator__learning_rate':[0.1,0.3,0.4]
}

models = { 
    'OVR_XGB': pipeline
}

params = { 
    'OVR_XGB': [parameters]
}

helper = EstimatorSelectionHelper(models, params)
helper.fit(X_train, y_train, n_jobs=-1, scoring='f1_macro')
model_result = helper.score_summary(sort_by='min_score')
model_result

We can see that the best **learning rate** value is **0.3**, it gives the best cross validation score.

In [ ]:
helper.grid_searches['OVR_XGB'].best_params_

In [ ]:
y_proba = helper.grid_searches['OVR_XGB'].predict_proba(X_test)
y_proba = np.array(y_proba)

Finaly, if we observe the model is correctly optimized reducing at the best the overfit.
Let's try to optimize the test_score, specifiy a threshold of probability of the prediction of the XGB model.

In [ ]:
from sklearn.metrics import f1_score
#label_ranking_average_precision_score
threshold = [0.05,0.1,0.4,0.5]
score = []
for thresh in threshold:
    y_predict = np.zeros((y_proba.shape[0], y_proba.shape[1]))
    for i in range(0, y_proba.shape[0]):
        for j in range(0, len(y_proba[i,:])):
            #y_predict[i,j] = 0
            if (y_proba[i,j] > thresh):
                y_predict[i,j] = 1
    score.append(f1_score(y_predict, y_test, average='macro'))
score

A threshold equal to **0.4** gives the best test score.

# 3 Select final supervised-model

From the previous job completed, we have the following results:
- LinearSVC model:
    - Training score: 0.77
    - Cross validation score: 0.69
    - Test validation score: 0.70
    - % overfit: 11%
- Random Forest model:
    - Training score: 0.81
    - Cross validation score: 0.72
    - Test validation score: 0.71
    - % overfit: 12%
- XGBoost model:
    - Training score: 0.84
    - Cross validation score: 0.73
    - Test validation score: 0.74
    - % overfit: 15%

We choose to select **Random Forest** classifier giving best test validation score result with lowest overfit percent.